# Amazon Reviews Competition

## Big Data Analytics

#### Group 13:

*Vincent Ott <br>
Lisa Wittmann <br>
Sophia Lorenz*

**October 5, 2023 <br>
University of Amsterdam**

## *Table of Contents*

[1. Libraries](#section-one) <br>

[2. Introduction](#section-two) <br>

[3. Raw Data](#section-three) <br>

[4. Data Preprocessing](#section-four) <br>

[5. Training and Test Data Split](#section-five) <br>

[6. Tokenization](#section-six) <br>

[7. Feature Engineering](#section-seven) <br>

[8. Redunant Variables](#section-eight) <br>

[9. Sparse Matrices](#section-nine) <br>

[10. Model Fitting](#section-ten) <br>
- [Shrinkage Methods](#subsection-ten-one) <br>
    - [Ridge Regression](#subsection-ten-one-one) <br>
    - [Lasso](#subsection-ten-one-two) <br>
- [Dimension Reduction Methods](#subsection-two-one) <br>
    - [Principal Components Regression](#subsection-two-one-one) <br>
    - [Partial Least Squarews](#subsection-two-one-two) <br>

[11. Model Evaluation](#section-eleven) <br>

[12. Model Comparison](#section-twelve) <br>

[13. Predictions on Test Data](#section-thirteen) <br>

[14. Submission](#section-fourteen) <br>

[15. Discussion](#section-fivteen) <br>

[16. References](#section-sixteen) <br>

[17. Task Division](#section-seventeen) <br>

<a id="section-one"></a>


## 1 Importing Packages

In [ ]:
library(tidyverse)
library(tidytext)
library(doMC)
library(pls)

<a id="section-two"></a>
## 2 Introduction

In this competition we will predict customer satisfaction regarding [Baby products purchased on Amazon.com](http://jmcauley.ucsd.edu/data/amazon/), based on their written reviews. The probability that a customer is satisfied is defined as rating > 3. 

### 2.1 Task and Goal

We built binary classifier that recognize customer satisfaction from textual reviews of baby products. Reviews were collected by Amazon on their webpage with a maximum satisfaction score of 5. We will define each customer review larger than three stars as satisfied and every customer review scoring three or smaller as unsatisfied. The perfomance will be measuerd by evaluating the area under the curve (AUC) of the receiver operating curve (ROC).


### 2.2 Dataset

The dataset origins from reviews on Amazon.com (http://jmcauley.ucsd.edu/data/amazon/). It contains 82.83 million unique reviews, from approximately 20 million users over a timespan of 19 years (May 1996 - 2014). The dataset contains 9.35 million items. Satisfaction was measured on on a 5-point Likert scale with 5 stars being greatly satisfied. To what extent will our models and results be generalizable? Two things are important to note. First, the data is quite outdated. It is from the very beginnings of e-commerce until 2014. Since 2014 it is also likely, that there have been major changes in e-commerce. For example, Amazon customers that ordered baby products in 2004 are probably a different demographic compared to customers in 2024. Especially if we consider that Amazon's core business used to be books. It was probably less common to order baby products from Amazon in 2004 than it will be in 2024. So, the language and satisfaction in Amazon customers might be different in 2024. The second aspect to consider about the generalizability is that the dataset only contains reviews for baby products. It is therefore unlikely that the models will hold for products such as razors or fantasy novels: "These diapers just saved my day" vs. "I read non-stop for 10 hours". In sum, our models and results should only generalize to a limited degree. Especially if we consider that there is more contemporary data readily available (which is outside the scope of the current competition).


### 2.3 Candidate Machine Learning Models and Features

The classification task on hand deals with a binary classification problem - customers being satisfied vs. unsatisfied. Furthermore, we are working with text and therefore will face a lot of features. Therefore our predictions of customer satisfaction were based on the following models: 

We used logistic regressions using shrinkage methods. Using shrinkage methods, i.e. constraining / regularizing the coefficient estimates such that the coefficient estimates are shrinked towards zero are beneficial as they result in sparser models (James et al., 2021). Thus, the following was applied:
- Ridge Regression <br>
- Lasso Regression <br>

We also applied models using dimension reduction methods. Reduction methods transform predictors and fit least squares models on the transformed variables. This way the regression problem is reduced from estimating all parameters p + 1 coefficients to linear combinations M + 1 predictors, resulting in reduced dimensions of the model (James et al., 2021). 
- Principal Component Regression <br>
- Partial Least Squares <br>

We chose the following features to include in those models: 
- **document occurence**: We encoded each token as either being present (1) or being not present (0) in a document.
- **token counts**: We created a document-by-term matrix which contains each token and its corresponding count within the document.
- **term frequency**: We calculated the relative frequency of a term within a document
- **inverse document frequency**: We calculated the inverse of the relative frequency with which a term occurs among all documents, expressed on a log scale (a measure of 'surprise')
- the **product of TF and IDF**


### 2.4 Bayes' error bound

The performance measure in this competition is AUC. On the other hand, the literature on sentiment analysis of Amazon reviews typically reports accuracy as a performance measure. However, accuracy and AUC are closely related and we will also not only look at the AUCs of our models but also at the accuracies. Thus, the accuracies serve as good indicators of lower Bayes' error bounds:

Haque et al. (2018) used a TF-IDF approach like us and classified reviews in positive (>3) and negative (<3). The cross-validated accuracies with logistic regression reached from 81.99% to 91.43%. The highest accuracy came with a linear support vector machine at 94.02%.

Given the limited scope of our project, an accuracy of 80% to 90% would therefore already be a good performance. The Bayes' error bound, however, is at least 94.02%.

<a id="section-three"></a>
## 3 Raw Data

The dataset consists of:
- a dataset containing the reviews on Amazon regarding baby products. The dataframe contains both, the trainind data and test data. The test data are the reviews for which the rating is missing for which we will provide a prediction. 
- a test dataset containing the reviews on Amazon regarding baby products.
The data are stored in csv (.csv) files. 

In [ ]:
# data attached to this notebook
list.files(path = "../input")

In [ ]:
# locate and load the data into memory
dir("../input", recursive = TRUE)

In [ ]:
# find the right file path
csv_filepath = dir("..", pattern="amazon_baby.csv", recursive=TRUE, full.names = TRUE)

# read in the csv file
amazon = read_csv(csv_filepath) %>%
    rownames_to_column('id') 

The dataframe contains the product `name`, the textual `review`, and the `rating`:

In [ ]:
head(amazon)

To get an idea how many products / datapoints each rating category has, we counted the ratings per rating category.

In [ ]:
# count per rating category
amazon %>% group_by(rating) %>% summarize(n = n())

The counts indicate that there are much fewer products / datapoints with low ratings as compared with high ratings. There are n = 300000 products / datapoints without a rating (`NA`). These serve as test data.

<a id="section-four"></a>
## 4 Data Preprocessing

### 4.1 Pasting `name`and `review` into a single string

Since baby products differ on quality which will cause the ratings to differ, we included product identity as a predictive feature. To do so, we prepended the product name to the review text.
By not handling product names separately, we also have at least the name of the reviews that are empty strings and can predict the ratings of such reviews. 

In [ ]:
# paste the `name` string and `review` string into a single string by "–"
amazon = amazon %>% 
    unite(review, name, review, sep = " — ", remove = FALSE)

head(amazon)

### 4.2 Satisfaction Split

The aim of this competition is to predict the probability that a customer is ***satisfied***. This is deemed to be the case if `rating > 3`.  Hence, we need a dependent variable `y` a factor that specifies whether this is the case. Therefore, we added an extra column to the dataframe categorizing each review into satisfied (1) or unsatisfied (0). 

In [ ]:
# Extra column to define staisfied vs. unsatisfied customers
# binary problem for customer rating larger than 3
amazon <- amazon %>%
  mutate(satisfied = case_when(
    is.na(rating) ~ NA,       # When rating is NA, satisfied should be NA
    rating <= 3 ~ 0,          # When rating is <= 3, satisfied should be 0
    rating > 3 ~ 1            # When rating is > 3, satisfied should be 1
  ))
head(amazon)

We will need a dependent variable `y` (customer satisfaction) as a factor that specifies whether customers ratings is larger than 3. This dependent variable `y`, namely customer satisfaction got factorized.

In [ ]:
# Factorize satisfied column
amazon$satisfied <- factor(amazon$satisfied)
str(amazon)

<a id="section-five"></a>
## 5 Training and Test Data Split

The logical index variable was applied to select the desired rows without the need to split the data frame into seperate sets. This way feature extraction will become easier. Data was split by selecting datapoints containing `NA`'s in the `satisfied` column as test data for which we want to make predictions. The remaining datapoints containing data in the `satisfied` column serves as training data. There are 153.531 training samples and 30.000 test samples.

In [ ]:
# count of training data and test data (contains `NA`s)
trainidx = !is.na(amazon$rating)
table(trainidx)

In [ ]:
# count of training data
amazon_train = amazon %>% filter(trainidx)
amazon_train %>% nrow()

# training dataframe
amazon_train %>% head(3)

In [ ]:
# count of test data
amazon_test = amazon %>% filter(!trainidx)
amazon_test %>% nrow()

# Number of NA's in `satisfied`
sum(is.na(amazon$satisfied))

# test dataframe
amazon_test %>% head(3)

The test data contains `NA` values in the `rating`and `satisfied` column.

### 5.1 Sample Train Data from original Dataframe

We are working with a vast amount of data, so called big data. So, for working in this workbook, it is advisable to only work with a subset of`amazon_train`. This makes the computation time more feasible, especially when training and testing our features and the model fit. However, for the final predictions we sampled the full set of `amazon_train` and thus make use of all the available training data.

In [ ]:
# set.seed(4)
# start with smallish sample: e.g. 5000
# for you final model use all train data: 153531
sampled_amazon_train = amazon_train %>% sample_n(5000) 
sampled_amazon_train %>% nrow()

### 5.2 Merge `sampled_amazon_train` and `amazon_test`

We merged again the sample training dataset and test dataset before creating features.

In [ ]:
# merging the sample training set and test set
amazon_merged = rbind(sampled_amazon_train, amazon_test)

head(amazon_merged)

In [ ]:
# counts of the training data and test data (contraining NA`s)
merged_trainidx = !is.na(amazon_merged$rating)
table(merged_trainidx)

It can be seen that the training dataset now only consists of a subsample with n = 5000 whereas the test data still consists of the full set of test data (n = 30000). For the final predictions we made use of the full training dataset consisting of n = 153531 datapoints.

<a id="section-six"></a>
## 6 Tokenization

We used `tidytext` to break up the text into separate tokens and count the number of occurences per review. Tokens are single words (unigrams), pairs of words called bi-grams, or groups of words, n-grams. 

### 6.1 Unigrams

Unigrams are single tokens (words) in the review text. They are the simplest and most basic form of text representation, where each token is considered as a separate unit. Unigrams are computationally efficient. We use the unigrams to prepare the text data for the following analysis. Further, used the unigrams to identify and analyze the sentiment or emotional tone of a piece of text.

In [ ]:
tokenize_unigrams = . %>% 

   # tokenize reviews at word level
   unnest_tokens(token, review) %>%

   # keep id (row number) in the result
   # keep satisfied (predictor) in the result
   # product names are included in the tokens
   # count tokens within reviews as 'n'
   count(id, satisfied, token)

In [ ]:
# create a unigram
amazon_merged %>% head() %>% tokenize_unigrams() %>% head()

As can be seen, in a unigram the token consists of a single word.

### 6.2 Bigrams

Bigrams are sequences of two consecutive tokens (words) that appear together in the review text. Bigrams consist of pairs of words that occur one after the other in a given order within a review. Therefore, they can provide insights into which pairs of words commonly co-occur in a text which is valuable information for capturing some level of context and relationship between words. We might use the bigrams for a sentiment analysis of the review texts.

In [ ]:
tokenize_bigrams = . %>%

   # tokenize reviews at word level
   unnest_tokens(token, review, token = "ngrams", n = 2) %>%
   
   # keep id (row number) in the result
   # keep satisfied (predictor) in the result
   # the two product names are included in each token column
   # count tokens within reviews as 'n'
   count(id, satisfied, token)

In [ ]:
# create a bigram
amazon_merged %>% head() %>% tokenize_bigrams() %>% head()

As can be seen, in a bigram the token consists of two words.

### 6.3 Stopwords

A problem of $TF_{d,t}$ is that it does not take into account that certain words simply occur more frequently because of their role in language (such as 'a', 'but', etc.). We removed such words that we theorized to be unimportant for the sentiment analysis.  Words such as 'is', 'a', 'the', etc, putatively do not carry a lot of information for the textual analysis and occur quite frequently in the review texts. Given that their usage was assumed to not vary a lot across satisfied vs. unsatisfied review texts, we decided to remove stopwords as defined in a customised stopwords list.
Whether those potential features are indeed uninformative with respect to the target variable was tested statistically.

As stopwords do not make a big difference, we decided not to include them.

In [ ]:
#stopwords = get_stopwords() 
#head(stopwords)

# remove stopwords
#amazon_merged_ <- 
 #   amazon_merged_1 %>%
    
    # stopwords are applied per word
  #  anti_join(stopwords, by = c("word" = "token"))

# peek at the result
#head(amazon_merged_1)

<a id="section-seven"></a>
## 7 Feature Engineering

Features computed for tokens in text are based on the Bag of Words (BoW) model (a bag of Tokens). In this approach each document is considered a bag of words, in which order is disregarded.

We used the following token features in the model: 

- **document occurence**: We encoded each token as either being present (1) or being absent (0) in a document. The presence or absence of specific words or terms can help identify patterns, similarities, or differences between documents.
- **token counts $n_{t,d}$**: We created a document-by-term matrix which contains each token $t$ and its corresponding count $n_{t,d}$ within a document $d$.
- **term frequency ($TF_{d,t}$)**: We calculated the relative frequency of a term within a document $\displaystyle {n_{d,t} \over  \sum_t n_{d,t}}$. The motivation for $TF_{d,t}$ is simply that the more often a token $t$ occurs in a document, the more likely it is that the topic of the document is closely related to that token. 
- **inverse document frequency $IDF_t$**: We calculated the inverse of the relative frequency with which a term occurs among all $N$ documents, expressed on a log scale (a measure of 'surprise') as $-\log\left({DF_t \over N}\right)$. Here $DF_t$ is the number of documents that contain the token $t$. The motivation for the $IDF_t$ is that the more wide spread the use of a token $t$ is among all documents, the less likely it conveys information about the topic of any particular document. Hence, the more surprising a word is, the more likely it conveys information about the topic of the document in which it is found. $IDF_t$ simply scales the $TF_{d,t}$ features accross documents. *This scaling may have an effect on scale sensitive algorithms like PCA and algorithms that rely on Euclidean distances such as kNN.*
- the **product of term frequency TF and inverse document frequency IDF $TFIDF_{d,t}$**: This quantifies the importance of a term for a given document. 
- the **average word length**: This quantifies the average length of a word.
- the **word count per review**: This quantifies the count of words per review assuming that unsatisfied reviews tend to have more words.
- **Unigrams**: Unigrams are single tokens (words) in the review text. They are the simplest and most basic form of text representation, where each token is considered as a separate unit. Unigrams are computationally efficient.
- **Bigrams**: Bigrams are sequences of two consecutive tokens (words) that appear together in the review text. Bigrams consist of pairs of words that occur one after the other in a given order within a review. Therefore, they can provide insights into which pairs of words commonly co-occur in a text which is valuable information for capturing some level of context and relationship between words.

For those features we created helper functions.

### 7.1 Document occurence

Document occurrence refers to a measure that indicates whether a word appears in a given review (document) within a collection of reviews (documents). Document occurrence is a binary measure, meaning it indicates whether the term is present (1) or absent (0) in a review (document). It is a basic form of term frequency representation in text analysis.

In [ ]:
# document occurence
amazon_merged %>% tokenize_unigrams() %>% filter(n == 0) %>% head(3)

As of yet, for each id there are only tokens which also occur. Thus, there are no non-occuring tokens for each id.

### 7.2 Token counts

The token counts were already implemented in column `n`.

### 7.3 Term frequency $TF_{d,t}$, Inverse document frequency $IDF_t$, and Product $TFIDF_{d,t}$

This helper function was built to calculate term frequency $TF_{d,t}$, the inverse document frequency $IDF_t$, and the product of term frequency and inverse document frequency $TFIDF_{d,t}$.

In [ ]:
# define a function using dplyr pipe
to_tf_idf = . %>% 

    # compute TF·IDF for each word per sentence
    bind_tf_idf(token, id, n) %>% 

    # words that are not present in a particular scentence are NA's but should be 0
    replace_na(list(tf = 0, idf = Inf, tf_idf = 0))

### 7.4 Average word length in unigrams

We calculated the mean of word length across all reviews.

In [ ]:
# Calculate average word length of unigram tokens and bind to a dataframe
avg_word_length <- . %>%
  
  # Calculate length of each word
  mutate(word_length = nchar(token)) %>%
  
  # Group by ID and calculate average word length
  group_by(id) %>%
  summarise(avg_word_length = mean(word_length, na.rm = TRUE))

### 7.5 Word count per Review

We calculated the word count per review (`ID`). 

In [ ]:
# Word count per review 
word_count <- . %>% 

  # group by review id
  group_by(id) %>%

  # count the number of words
  summarise(word_count = sum(n()))

### 7.6 Unigrams

Unigrams were created which contain the token, the token count, the term frequency `$TF_{d,t}$`, the inverse document frequency `$IDF_t$`, and the product of the two `$TFIDF_{d,t}$`, respectively. Further, we included the mean word length per review (`avg_word_length`) and the word count (`word_count`).

In [ ]:
# create unigrams 
unigrams_df <- amazon_merged %>% tokenize_unigrams() %>% to_tf_idf()


# Calculate average word length of unigram tokens and bind to unigrams_df
avg_word_length <- unigrams_df %>%
  
  # Calculate length of each word
  mutate(word_length = nchar(token)) %>%
  
  # Group by ID and calculate average word length
  group_by(id) %>%
  summarise(avg_word_length = mean(word_length, na.rm = TRUE))


# Bind avg_word_length to unigrams_df
unigrams_df <- unigrams_df %>%
  left_join(avg_word_length, by = "id")



# Word count per review 
word_count <- unigrams_df %>% 

  # group by review id
  group_by(id) %>%

  # count the number of words
  summarise(word_count = sum(n()))


# Bind word_count to unigrams_df
unigrams_df <- unigrams_df %>%
  left_join(word_count, by = "id")


tail(unigrams_df)
dim(unigrams_df)

As a safeguard, we use a boxplot to visualize the distribution of "surprise" that the tokens come with.

In [ ]:
# boxplot of the inverse document frequency `idf` of the unigram data
boxplot_idf <- unigrams_df %>%
  ggplot(aes(x = 1, y = idf)) +  # Using x = 1 to create a single boxplot
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of IDF (Inverse Document Frequency)",
       x = "All reviews", y = "IDF (Inverse Document Frequency)") + 
  theme(legend.position = "none") +
  theme_minimal()

boxplot_idf

We should see that the vast majority of tokens lies between 1 and 4.

Next, we created a boxplot to visualise the average word length distribution.

In [ ]:
# boxplot of the average word length distribution
plot_reviews_awl <- unigrams_df %>%
  ggplot(aes(x = 1, y = avg_word_length)) +
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of average word length",
       x = "All reviews", y = "Average word length") + 
  theme(legend.position = "none") +
  theme_minimal()

plot_reviews_awl

We should see that the average word length is quite spread across amazon reviews with the most average word length value being between 4 and 5.

Finally, we implemented a boxplot to visualize the word count.

In [ ]:
# boxplot of the word_count distribution
plot_word_count <- unigrams_df %>%
  ggplot(aes(x = 1, y = word_count)) +
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of word_count",
       x = "All reviews", y = "Word count") + 
  theme(legend.position = "none") +
  theme_minimal()

plot_word_count

We should see that the word count is widely spread across amazon reviews with the majority of word counts being between 50 and 110 words.

### 7.7 Bigrams

Bigrams were created which contain the token, the token count, the term frequency `$TF_{d,t}$`, the inverse document frequency `$IDF_t$`, and the product of the two `$TFIDF_{d,t}$`, respectively. Further, we included the mean word length per review (`avg_word_length`) and the word count (`word_count`).

In [ ]:
# create bigrams 
bigrams_df <- amazon_merged %>% tokenize_bigrams() %>% to_tf_idf()


# Calculate average word length of bigram tokens and bind to bigrams_df
avg_word_length <- bigrams_df %>%
  
  # Calculate length of each word
  mutate(word_length = nchar(token)) %>%
  
  # Group by ID and calculate average word length
  group_by(id) %>%
  summarise(avg_word_length = mean(word_length, na.rm = TRUE))


# Bind avg_word_length to the bigrams_df dataframe
bigrams_df <- bigrams_df %>%
  left_join(avg_word_length, by = "id")


# Word count per review 
word_count <- bigrams_df %>% 

  # group by review id
  group_by(id) %>%

  # count the number of words
  summarise(word_count = sum(n()))


# Bind word_count to bigrams_df 
bigrams_df <- bigrams_df %>%
  left_join(word_count, by = "id")


tail(bigrams_df)
dim(bigrams_df)

We plotted the distribution of "surprise" that the tokens come with using the `idf`.

In [ ]:
# boxplot of the inverse document frequency `idf` of the bigram data
boxplot_idf <- bigrams_df %>%
  ggplot(aes(x = 1, y = idf)) +  # Using x = 1 to create a single boxplot
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of IDF (Inverse Document Frequency)",
       x = "All reviews", y = "IDF (Inverse Document Frequency)") + 
  theme(legend.position = "none") +
  theme_minimal()

boxplot_idf

We should see that the vast majority of tokens has an IDF between 1 and 4.

Next, we created a boxplot to visualise the average word length distribution.

In [ ]:
# boxplot of the average word length distribution
plot_reviews_awl <- bigrams_df %>%
  ggplot(aes(x = 1, y = avg_word_length)) +
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of average word length",
       x = "All reviews", y = "Average word length") + 
  theme(legend.position = "none") +
  theme_minimal()

plot_reviews_awl

We should see that the average word length is quite spread across amazon reviews with the most average word length value being between 4 and 5.

Finally, we implemented a boxplot to visualize the word count.

In [ ]:
# boxplot of the word_count distribution
plot_word_count <- bigrams_df %>%
  ggplot(aes(x = 1, y = word_count)) +
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of word_count",
       x = "All reviews", y = "Word count") + 
  theme(legend.position = "none") +
  theme_minimal()

plot_word_count

We should see that the word count is widely spread across amazon reviews with the majority of word counts being between 50 and 110 words.

### 7.8 Sentiment Analyses

We used a helper function in order to implement the libraries for the sentiment analysis.

In [ ]:
# helper function to get a lexicon library
get_lexicon <- function(lexicon_name = names(textdata:::download_functions)) {
    lexicon_name = match.arg(lexicon_name)
    textdata:::download_functions[[lexicon_name]](".")
    rds_filename = paste0(lexicon_name,".rds")
    textdata:::process_functions[[lexicon_name]](".",rds_filename)
    readr::read_rds(rds_filename)
}

#### 7.8.1 Sentiments with NRC library

We opted for the NRC Sentiment Vocabulary due to its comprehensive array of emotions for discerning the sentiment within a given review. This vocabulary encompasses a spectrum of emotional states, including anger, anticipation, disgust, fear, joy, sadness, surprise, confidence, and both negative and positive sentiments. The quantity of these identified emotions within a review should effectively convey the prevailing mood. Utilizing this diversity of emotions allows us to more precisely differentiate between negative and positive states, enhancing our ability to recognize various expressions such as frustration. 

In [ ]:
# load NRC word library
nrc <- get_lexicon('nrc')

# NRC scores for unigrams
nrc_df_uni <- unigrams_df %>%
    inner_join(nrc, by = c("token" = "word"), copy = TRUE, relationship = "many-to-many") %>%
    group_by(id, sentiment) %>% 
    mutate(sentiment = ifelse(is.na(sentiment), "nrc_", paste0("nrc_", sentiment)))

head(nrc_df_uni)
dim(nrc_df_uni)

We also aimed to apply the nrc on bigrams as we assumed better predictions of the sentiment analysis. However, this was computationally expensive and was therefore not implemented.

In [ ]:
# NRC scores for bigrams
# nrc_df_bi <- bigrams_df %>%
  # separate(bigrams_df, into = c("word1", "word2"), sep = " ") %>%
  # inner_join(unigrams_df, by = c("word1" = "word"), copy = TRUE, relationship = "many-to-many") %>%
  # group_by(id, sentiment) %>% 
  # mutate(sentiment = ifelse(is.na(sentiment), "nrc_", paste0("nrc_", sentiment)))
  
# head(nrc_df_bi)
# dim(nrc_df_bi)

#### 7.8.2 Sentiments with BING library

Alternatively, the BING library which assigns 'positive' or 'negative' to a token of the unigram / bigram. We expected words associated with positive sentiments to occur frequently in positive reviews and such with negative sentiments in negative reviews.

Note: code only works for unigrams. Would be interesting to see how predictions perform on bigrams. We assumed the predictions to get even better.

In [ ]:
# load bing word library
bing <- get_sentiments('bing') 

# assignment of negative 
bing_df_uni <- unigrams_df %>%
    inner_join(bing, by = c("token" = "word"), copy = TRUE, relationship = "many-to-many") %>%
    group_by(id, sentiment) %>% 
    mutate(sentiment = ifelse(is.na(sentiment), "bing_", paste0("bing_", sentiment)))

head(bing_df_uni)
dim(bing_df_uni)

### Features Dataframe

We combined the features into one dataframe. The dataframe contains the following features: 
- the token count
- the term frequency `$TF_{d,t}$`
- the inverse document frequency `$IDF_t$`
- the product of the two `$TFIDF_{d,t}$`, respectively
- mean word length per review (`avg_word_length`)
- word count (`word_count`)


First, we combined the unigrams with bigrams.

In [ ]:
# combining the unigrams and bigrams
# unigrams_bigrams_combined = rbind(nrc_df_uni, bing_df_uni)
# features_df = unigrams_df %>% 
    # left_join(bigrams_df, by = "id", relationship = "many-to-many")

# head(unigrams_bigrams_combined)

# sum(nrow(unigrams_df), nrow(bigrams_df)) == nrow(unigrams_bigrams_combined)

Unfortunately, we were not able to join the unigram containing sentiments with the bigrams containing sentiments due to high computational expenses.

In [ ]:
# combining the unigrams and bigrams
# nrc_bing_combined = rbind(nrc_df_uni, bing_df_uni)
# nrc_bing_combined = nrc_df_uni %>% 
    # left_join(bing_df_uni, by = "id", relationship = "many-to-many")

# head(nrc_bing_combined)
# dim(nrc_bing_combined)

Unfortunately, we were not able to join the unigram containing nrc sentiments with the unigrams containing bing sentiments due to high computational expenses.

In [ ]:
# combining the unigrams with the bigrams both containing sentiments
# features_df = rbind(nrc_df_uni, nrc_df_bi)
# features_df = unigrams_df %>% 
    # left_join(bigrams_df, by = "id", relationship = "many-to-many") %>% 

# head(features_df)

We ended up using the unigrams_df upon which we trained our model. This model was computationally the least expensive while still containing valuable features.

In [ ]:
features_df = unigrams_df

<a id="section-eight"></a>
## 8 Redunant Variables

Redunant variables refer to variables which have: 
* Near-zero variance
* High correlation
* Multi-collinearity

Redunant variables can be removed from the training dataset because they do not contribute meaningful information to the analysis and do not help discriminate between observations. By removing such variables, we simplify our model, which can lead to more interpretable results and understandable models. 

Moreover, having too many features or variables, especially those with low or no variance, can lead to overfitting. By removing uninformative variables, we reduce the risk of overfitting and improve the model's generalization to new data. 

Finally, redunant variables increase the computational cost of modeling and analysis without providing any real benefit. Removing them can speed up the processing time and reduce memory usage (James et al., 2021).

### 8.1 Non-zero variance features

Variables with near-zero variation are variables with mostly one and the same value. Variables with no or very little variation can be removed from the training dataset because they are essentially constant and do not help discriminate between observations.

In order to check for features with near zero variance, we decided not to use the `nearZeroVar` function of the `caret` package' because it is computationally ineffective.

For binary and count data as is the case here the variance is determined by the average. Hence, for the current data we accounted for document frequencies.


### 8.2 Highly correlated features

High correlation between features indicate they provide same or very similar information. This leads to instability of coefficient estimates (collinearity or inestimability in the worst cases). In order to correct for these highly correlated variables one would actually remove one of the two correlated features.

Although correlated (linear combinations of) features may exist in this dataset, it would we computationally too cumbersome to remove them with those vasts amounts of data. Nevertheless, ridge regression and lasso inherently allow to disregard a specific measure to control for highly correlated features.


### 8.3 Multicollinearity

Multicollinearity refers to the situation where two or more variables are highly correlated, meaning that highly correlated linear combinations of features exist. In fact, those variables can be exactly predicted from other features. 

We would technically have to control for multicollinearity but regularization techniques will do this inherently.


### 8.4 Low frequency tokens

We theorized that tokens with a very low frequency were either idiosyncratic strings of miss-spellings that occur only in singular reviews. Therefore, tokens that occured in less than 0.01% of the documents were also disregarded in the model fitting. This allowed to remove tokens that appeared in less than 18 reviews (see below).

$$IDF_t = -\log\left({\text{df}_t \over N}\right) = -\log(\text{proportion of document in which }t\text{ occurs})$$ 

we can filter the rows in `features_df` for which $-\log(\text{df}_t / N) \leq -\log(0.01\%)$ (i.e., the 'surprise' should be lower than $-\log(0.01/100)$).

In [ ]:
# Approximately 180,000 reviews in the data set
0.0001 * 180000

This shows that it requires at least 18 reviews per document in order to be included in the analysis if we were to take a boundary of 0.01%.

In [ ]:
cut_off = -log(0.01/100)
print(cut_off)

As a safeguard, we use a boxplot to visualize the distribution of "surprise" that the tokens come with. We should see that the vast majority of tokens should lie below the cutoff and remain included in the next step.

In [ ]:
# boxplot of the inverse document frequency `idf` of the features_df
boxplot_idf <- features_df %>%
  ggplot(aes(x = 1, y = idf)) +  # Using x = 1 to create a single boxplot
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of IDF (Inverse Document Frequency)",
       x = "All reviews", y = "IDF (Inverse Document Frequency)") + 
  theme(legend.position = "none") +
  theme_minimal()

boxplot_idf

In [ ]:
# Keep only tokens that appeared in 18 reviews or more
nrow(features_df)
features_df = features_df %>% filter(idf <= cut_off)
nrow(features_df)

After excluding those tokens with a very low frequency, the distribution of "surprise" in the tokens looks as follows.

In [ ]:
# boxplot of the inverse document frequency `idf` of the features_df
boxplot_idf <- features_df %>%
  ggplot(aes(x = 1, y = idf)) +  # Using x = 1 to create a single boxplot
  geom_boxplot(fill = "blue") + 
  labs(title = "Distribution of IDF (Inverse Document Frequency)",
       x = "All reviews", y = "IDF (Inverse Document Frequency)") + 
  theme(legend.position = "none") +
  theme_minimal()

boxplot_idf

<a id="section-nine"></a>
## 9 Sparse Matrices

For word counts from longer documents we get huge matrices. This causes problems because large matrices take a lot of computer memory and the computation speed will be quite low (Van de Meer et al., 2023). In order to avoid that we used sparse matrices. Sparse matrices only store non-zero elements (triplets: row, col, value). This is efficient because most entries in the text analytics matrices are equal to 0 anyways.

In [ ]:
# Turn features into a sparse design matrix 
X = features_df %>% 
    cast_sparse(id, token, tf_idf) %>% 
    # Remove rows that do not belong to cases
    .[!is.na(rownames(.)),]

# Verify the result:
X[1:8,20:25]
cat("rows, columns: ", dim(X))

In [ ]:
# Look at the memory footprint
print(object.size(X), unit="Mb")

In [ ]:
# Split design matrix into training and test portions
X_train <- X[rownames(X) %in% amazon_merged[!is.na(amazon_merged$satisfied),]$id,]
X_test  <- X[rownames(X) %in% amazon_merged[is.na(amazon_merged$satisfied),]$id,]

In [ ]:
# Target variable
y_train <-
    # Order of the cases in `X` may not be the same anymore as in `sampled_amazon`,
    # hence we line up rownames in X with `id` in `sampled_amazon`
    data.frame(id = rownames(X)) %>% 
    inner_join(sampled_amazon_train, by = "id") %>% 

    # Extract 'satisfied'
    pull(satisfied) %>%
    as.factor()

<a id="section-ten"></a>
## 10 Model Fitting

Since we do not have any knowledge about which features are mostly predictive in customer satisfaction we need to apply a process of trial and error. Though, our data is big resulting in thousands of candidate features such that automation to balance flexibility and predictive performance of this process becomes essential.

In order to control the flexibility of the models, a tuning parameter lamda needs to be adjusted such that the optimal predictive performance of a model can be found. The optimal size of lambda can be determined by using either a validation set, or using cross-validation. If lambda is zero, then the tuning does not have any effects on the model, it will simply estimate least squares in case of linear models. Larger values of lambda lead to more coeffiecients being drawn towards zero, thus, being not included in the model. This results in a less flexible model, leading to lower variance and increased bias (James et al., 2021).

The type of regularization that is used by `glmnet` is controled by the `alpha` parameter.  The amount of regularization is specified by means of the `lambda` parameter. To tune the lambda parameter cross-validation was applied with help of the `cv.glmnet()` function (Van de Meer, 2023).


<a id="subsection-ten-one"></a>
### 10.1 Shrinkage Methods

Shrinkage methods are techniques to shrink the regression coefficients towards zero. First, a model is fit that contains all predictors. Then, the coefficient estimates are constrained / regularized given the tuning parameter lambda to reduce the variance of the model (James et al., 2021). Models using shrinkage methods enable to take many features while automatically reducing redundant flexibility to any desired level.

<a id="subsection-ten-one-one"></a>
#### 10.1.1 Ridge regression
Ridge regression shrinks coefficient estimates that do not contribute much to the prediction near zero. This penalty is not applied to the intercept (James et al., 2021).

In [ ]:
# Speed up tuning by using all 4 CPU cores
doMC::registerDoMC(cores = 4)

In [ ]:
# Cross-validate the lambda parameter in ridge model
cv_fitridge <- glmnet::cv.glmnet(
    X_train,        # Training data (predictors)
    y_train,        # Response variable
    family = "binomial",    # Specify binary logistic regression
    parallel = TRUE,        # Use parallel processing if possible
    standardize = TRUE,     # Standardize the predictors
    type.measure = "auc",   # Type of measure to optimize (Area Under the ROC Curve)
    nfold = 5,              # Number of cross-validation folds
    alpha = 0               # Ridge regression (vs. Lasso)
)

# View the cross-validated Ridge logistic regression model
cv_fitridge

In [ ]:
# optimal value for the tuning parameter lambda
cv_fitridge$lambda.min
plot(cv_fitridge)

<a id="subsection-ten-one-two"></a>
#### 10.1.2 Lasso
Like ridge regression, the lasso forces a penalty on the coefficent estimates that do not contribute much to the prediction of the model so that they are drawn towards zero. But in contrast to ridge regression, the lasso shrinks some of the coefficient estimates, those that contribute the least to the prediction of the model, entirely to zero (James et al., 2021).

In [ ]:
# Cross-validate the lambda parameter in lasso model
cv_fitlasso <- glmnet::cv.glmnet(
    X_train,        # Training data (predictors)
    y_train,        # Response variable
    family = "binomial",    # Specify binary logistic regression
    parallel = TRUE,        # Use parallel processing if possible
    standardize = TRUE,     # Standardize the predictors
    type.measure = "auc",   # Type of measure to optimize (Area Under the ROC Curve)
    nfold = 5,              # Number of cross-validation folds
    alpha = 1               # Lasso regression
)

# View the cross-validated Lasso logistic regression model
cv_fitlasso

In [ ]:
# optimal value for the tuning parameter lambda
cv_fitlasso$lambda.min
plot(cv_fitlasso)

<a id="subsection-two"></a>
### 10.2 Dimension Reduction Methods

Dimension reduction methods transform predictors and fit a least squares model using the transformed variables. This way the regression problem gets reduced from estimating linear combinations for each predictor to estimating linear combinations for each dimensions (James et al., 2021).

<a id="subsection-ten-two-one"></a>
#### 10.2.1 Principal Components Regression

Principal components regression (PCR) is an unsupervised dimension reduction technique for regression where the first principal component is defined as the linear combination yielding the highest variance out of all possible linear combinations. It minimized the sum of squared distances between an observation datapoint and the regression line. The first principal component is chosen such that the regression line is as close as possible to the original observations. Thus, the first principal component will always contain the most information. The second principal component is a linear combination of variables that are unrelated to the variables of the first principal component. Thus, the second principal component is perpendicular / orthogonal to the first principal component. This principle is the same throughout all principal components. The number of princiap components can be chosen through cross-validation.

The assumption of PCR is that a small number of principal components suffices to explain variability in the data. In PCR the first principal components are constructed which are then used as predictors in a linear regression model (James et a., 2021).


In [ ]:
# set.seed(1)
# PCR with standardized predictors and cross-validation (10-fold cross-validation error for each possible number of principal components being used)
# pcr_fit <- pcr(satisfied ~., data = amazon_merged, scale = TRUE, validation = "CV", parallel = TRUE)
# summary(pcr_fit)

<a id="subsection-ten-two-two"></a>
#### 10.2.2 Partial Least Sqaures

Partial Least Squares (PLS) is a supervised alternative to PCR. 

We ended up not includind the dimension reduction techniques, because we are only allowed using the glmnet and pls packages, the former not supporting PCR/PLS and the latter not supporting sparse matrix formats, which made it difficult for us to implement dimensionality reduction methods.

<a id="section-eleven"></a>
## 11 Model Evaluation

Various predictive performance measures exist in order to evaluate the model.
The performance of our submission is evaluated using the ***area under the curve*** (**AUC**) of the *receiver operating curve* (*ROC*).

Other indicators that were used are prediction accuracy, AIC, BIC, and adjusted R square.

### 11.1 Performance of ridge regression

First, we evaluated the performance of ridge regression on the training dataset in regards to prediction accuracy. 

In [ ]:
# prediction accurary ridge regression
pred_ridge <- predict(
    cv_fitridge,             # Cross-validated Ridge logistic regression model
    X_train,                 # The training data (predictors)
    s = "lambda.min",        # Use the lambda value that minimizes the cross-validated error
    type = "class"           # Predict class labels
) %>% 
as.factor()                 # Convert the predictions to a factor (categorical variable)

In [ ]:
# confusion matrix ridge regression
confusion_cv_ridge <- caret::confusionMatrix(pred_ridge, y_train)

# Accuracy
mean(pred_ridge == y_train)

### 11.2 Performance of lasso

Second, we evaluated the performance of lasso on the training dataset in regards to prediction accuracy. 

In [ ]:
# prediction accurary lasso
pred_lasso <- predict(
    cv_fitlasso,             # Cross-validated Lasso logistic regression model
    X_train,                 # The training data (predictors)
    s = 'lambda.min',        # Use the lambda value that minimizes the cross-validated error
    type = 'class'           # Predict class labels
) %>% 
as.factor()                 # Convert the predictions to a factor (categorical variable)

In [ ]:
# confusion matrix lasso
confusion_cv_lasso <- caret::confusionMatrix(pred_lasso, y_train)

# Accuracy
mean(pred_lasso == y_train)

<a id="section-twelve"></a>
## 12 Model Comparison

We compared the fitted models, the ridge regression model and the lasso, with different performance evaluation techniques on the training data. This way we could determine which model to use for the final predictions on the test set.

In [ ]:
# Ridge
acc_cv_ridge  <- mean(pred_ridge == y_train)                  # Accuracy
auc_cv_ridge  <- max(cv_fitridge$cvm)                         # highest AUC value
sens_cv_ridge <- confusion_cv_ridge$byClass[["Sensitivity"]]  # Sensitivity
spec_cv_ridge <- confusion_cv_ridge$byClass[["Specificity"]]  # Specificity

# Lasso
acc_cv_lasso  <- mean(pred_lasso == y_train)                  # Accuracy
auc_cv_lasso  <- max(cv_fitlasso$cvm)                         # highest AUC value
sens_cv_lasso <- confusion_cv_lasso$byClass[["Sensitivity"]]  # Sensitivity
spec_cv_lasso <- confusion_cv_lasso$byClass[["Specificity"]]  # Specificity

# Table to see which model has highest accuracy, highest AUC value, Sensitity and Specificity
table <- data.frame(
  Model = c("Lasso", "Ridge"),
  AUC = c(auc_cv_lasso, auc_cv_ridge),
  Accuracy = c(acc_cv_lasso, acc_cv_ridge),
  Sensitivity = c(sens_cv_lasso, sens_cv_ridge),
  Specificity = c(spec_cv_lasso, spec_cv_ridge)
)
table


# Plot 1: AUC
plot_auc <- ggplot(table, aes(x = Model, y = AUC, fill = Model)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "AUC Comparison",
       x = "Model",
       y = "AUC") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_fill_brewer(palette = "Set1")

# Plot 2: Accuracy
plot_accuracy <- ggplot(table, aes(x = Model, y = Accuracy, fill = Model)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Accuracy Comparison",
       x = "Model",
       y = "Accuracy") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_fill_brewer(palette = "Set2")

# Plot 3: Sensitivity
plot_sensitivity <- ggplot(table, aes(x = Model, y = Sensitivity, fill = Model)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Sensitivity Comparison",
       x = "Model",
       y = "Sensitivity") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_fill_brewer(palette = "Set2")

# Plot 4: Specificity
plot_specificity <- ggplot(table, aes(x = Model, y = Specificity, fill = Model)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Specificity Comparison",
       x = "Model",
       y = "Specificity") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_fill_brewer(palette = "Set2")


gridExtra::grid.arrange(plot_auc, plot_accuracy, plot_sensitivity, plot_specificity, nrow = 2)


<a id="section-thirteen"></a>
## 13 Predictions on Test Data

Based on our model we made predictions on the test data. Those predictions are submitted in the competition.

In [ ]:
# Performance on test set
Pred_lasso = predict(
cv_fitlasso, X_test, s = 'lambda.min', type = 'response'
) %>% factor()

<a id="section-fourteen"></a>
## 14 Submission

In order to participate in the competition, the predictions had to be formatted and saved in a csv. file. The csv. file was submitted to the competition.

In [ ]:
# format predictions
predictions_df <- data.frame(
    Id = rownames(X_test),
    Prediction = Pred_lasso
)
predictions_df <- predictions_df %>% arrange(as.integer(Id))

head(predictions_df)

In [ ]:
# Write predictions data frame to file
write_csv(predictions_df, file="predictions.csv")

<a id="section-fifteen"></a>
## 15 Discussion

Task of this competition was to predict customer satisfaction based on their written reviews. Given that customer satisfaction is determined by a five-star system on amazon we were faced with a classification problem. 

Models we fitted on the training data included a ridge regression model and the lasso. Attempts to fit principal component regression models or a partial least squares model failed because the R function for fitting PCR and PLS, respectively do not allow for sparse matrices. However, fitting the PCR model and the PLS model to the vast amount of data we had would have been computationally expensive. 

Our first submission contained a simple lasso with only Term frequency *TF*, Inverse document frequency *IDF*, and Product *TFIDF* (Version 21). This resulted in a quite good prediction score of 0.81476.

We worked on features and a lasso regression model containing only unigrams which gave a predictive value of 0.93292.

So far, we have only tried lasso models, as they always had a better AUC value and specificity, but since the ridge model had better specificity (Lasso: 0.9607033; Ridge: 0.9769469) and almost similar accuracy (Lasso: 0.8990497, Ridge: 0.8931877), we were curious to try a submission with a ridge regression model as well. This resulted in a slightly worse prediction of 0.92579, whereas the previous version with Lasso had a value of 0.93292. Since our suspicion that Ridge regression might be a better model was refuted, we stuck with the Lasso model for subsequent runs. Furthermore, we hypothesize that the lasso model deals overfitting issues better the more features we add because coefficients that do not contribute much to the predictions are drawn to zero (James, et al., 2021). 

The submission of the first round contained a lasso regression model, where we used a bigram only. This resulted in a prediction score of 0.94951 meaning that the prediction score improved further.

For the second round of the competition we focused on sentiment analysis under the assumption that the sentiment of a review can also be a good indicator of a satisfied vs. unsatisfied customers based on their written review texts. Thus, we used unigrams conducted sentiment analyses. We used the NRC library for that. The NRC assigns positive or negative valences or one of the 8 basic emotions to the token. Given that we were to predict customer satisfaction (satisfied - unsatisfied) we also used the Bing library which assigns 'positive' or 'negative' to tokens. This way we could predict the positive vs. negative sentiment of a review (Class 3: Text Analytics - Big 5 from Text). Eventually, the sentiment analysis did not work due to problems with the length of the prediction file. 

Thus, for the final prediction our best AUC scores were achieved using bigrams with the selected features as described above.

<a id="section-sixteen"></a>
## 16 References

​
He, R. & McAuley, J.(2016). Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering. Amazon Product Data. WWW. Retreived from: http://jmcauley.ucsd.edu/data/amazon/
​

dan_vdmeer, Dave Leitritz, Joost van Kordelaar, Raoul. (2023). BDA 2023 Customer sentiment from reviews. Kaggle. https://kaggle.com/competitions/customer-sentiment-from-reviews-bda-2023

​
Grasman, R. (2018). Feature extraction from Signals. DropBox. Retrieved from: https://paper.dropbox.com/doc/Feature-extraction-from-Signals-qCp5uvj47gmyuw5nmB8lL

​
James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). An introduction to statistical learning (1st ed.). Springer.

​
Haque, T. U., Saber, N. N., & Shah, F. M. (2018, May). Sentiment analysis on large scale Amazon product reviews. In 2018 IEEE international conference on innovative research and development (ICIRD) (pp. 1-6). IEEE.

<a id="section-seventeen"></a>
## 17 Task Division
​
* **Vincent:** TFIDF, sparse matrix, bayes´ error bound, amazon_merged pipeline
* **Lisa:** Features, implementing models, comparisons
* **Sophia:** Features, writing and formatting
